# Image Classification

### Task
* Overftting을 피하며, accuracy를 높혀 보자
* Image size: 224 또는 299로 변경하여 수행 (baseline code는 `image_size`: 150)
* 밑에 제시된 여러가지 시도를 해보자

### Dataset
* [Google flower datasets](https://github.com/tensorflow/models/blob/master/research/inception/inception/data/download_and_preprocess_flowers.sh)
* 5개의 클래스(daisy, dandelion, roses, sunflowers, tulips)로 이루어진 꽃 이미지 데이터를 분류

### Baseline code
* Dataset: train, validation, test로 split
* Input data shape: (`batch_size`, 150, 150, 3)
* Output data shape: (`batch_size`, `num_classes`=5)
* Architecture: 
  * `Conv2D` (x3) - `Dense` - `Softmax`
  * [`tf.keras.layers`](https://www.tensorflow.org/api_docs/python/tf/keras/layers) 사용
* Training
  * `model.fit_generator` 사용
  * `tf.keras.preprocessing.image.ImageDataGenerator` 사용 for data augmentation
* Evaluation
  * `model.evaluate_generator` 사용 for test dataset

### Try some techniques
* Change model architectures (Custom model)
  * Or use pretrained models
* Data augmentation
* Various regularization methods

## Importing packages

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import time

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
tf.enable_eager_execution()

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

os.environ["CUDA_VISIBLE_DEVICES"]="0"

## Data Loading

이미지 분류 프로그램을 만들려면 먼저 데이터 세트를 다운로드해야합니다. 우리가 사용하는 데이터 세트는 Google Flower dataset 입니다. 먼저 dataset을 다운로드 한 다음 `'../../datasets/'` 디렉토리에 저장하고 압축을 풉니다.

데이터 구조는 아래와 같습니다.

<pre style="font-size: 10.0pt; font-family: Arial; line-height: 2; letter-spacing: 1.0pt;" >
<b>flower</b>
|__ <b>train</b>
    |____ <b>daisy</b>: [5547758_eea9edfd54_n.jpg, 5673551_01d1ea993e_n.jpg, ....]
    |____ <b>dandelion</b>: [7355522_b66e5d3078_m.jpg, 10443973_aeb97513fc_m.jpg, ...]
    |____ <b>...</b>: [...]
|__ <b>validation</b>
    |____ <b>daisy</b>: [705422469_ffa28c566d.jpg, 721595842_bacd80a6ac.jpg, ...]
    |____ <b>dandelion</b>: [7355522_b66e5d3078_m.jpg, 751941983_58e1ae3957_m.jpg, ...]
    |____ <b>...</b>: [...]
|__ <b>test</b>
    |____ <b>daisy</b>: [99306615_739eb94b9e_m.jpg, 813445367_187ecf080a_n.jpg, ...]
    |____ <b>dandelion</b>: [8181477_8cb77d2e0f_n.jpg, 8223949_2928d3f6f6_n.jpg, ...]
    |____ <b>...</b>: [...]
</pre>

In [ ]:
# I upload zip file on my dropbox
# if you want to download from my dropbox uncomment below
# !wget https://goo.gl/motrG4
# !mv motrG4 flower.zip
# !unzip flower.zip
# !mkdir ../../datasets
# !mv flower ../../datasets
# !rm flower.zip

In [ ]:
base_dir = '../../datasets/flower/'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

print(train_dir)
print(validation_dir)
print(test_dir)

In [ ]:
class_name = sorted(os.listdir(train_dir))
for name in class_name:
  print(name)

# Understanding out data

train, validation 및 test 디렉토리에서 얼마나 많은 이미지가 있는지 살펴 보겠습니다.

In [ ]:
num_train = 0
num_val = 0
num_test = 0
for name in class_name:
  train_path = os.path.join(train_dir, name)
  val_path = os.path.join(validation_dir, name)
  test_path = os.path.join(test_dir, name)
  print("Number of {} class: for train: {} / for validation: {} / for test: {}".format(name,
                                                                len(os.listdir(train_path)),
                                                                len(os.listdir(val_path)),
                                                                len(os.listdir(train_path))))
  num_train += len(os.listdir(train_path))
  num_val += len(os.listdir(val_path))
  num_test += len(os.listdir(test_path))

print('--------')
print("Total training images:", num_train)
print("Total validation images:", num_val)
print("Total test images:", num_test)

# Setting Model Parameters

For convenience, let us set up variables that will be later used while pre-processing our dataset and training our network.

In [ ]:
batch_size = 100
epochs = 15
IMG_SHAPE = 150  # Our training data consists of images with width of 150 pixels and height of 150 pixels

Images should be formatted into appropriately pre-processed floating point tensors before being fed into the network. The steps involving preparing these images are:

1. Read images from the disk
2. Decode contents of these images and convert it into proper grid format as per their RGB content
3. Convert them into floating point tensors
4. Rescale the tensors from values between 0 and 255 to values between 0 and 1, as neural networks prefer to deal with small input values.

Fortunately, all these tasks can be done using a single class provided in **tf.keras** preprocessing module, called **ImageDataGenerator**. Not only it can read images from the disks and preprocess images into proper tensors, but it will also set up generators that will turn these images into batches of tensors, which will be very helpful while training our network as we need to pass our input to the network in the form of batches.

We can easily set up this using a couple of lines of code.

In [ ]:
train_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our training data
val_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our validation data
test_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our test data

After defining our generators for training and validation images, **flow_from_directory** method will load images from the disk and will apply rescaling and will resize them into required dimensions using single line of code.

In [ ]:
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_dir,
                                                           # Its usually best practice to shuffle the training data
                                                           shuffle=True,
                                                           target_size=(IMG_SHAPE,IMG_SHAPE), #(150,150)
                                                           class_mode='categorical')

In [ ]:
val_data_gen = val_image_generator.flow_from_directory(batch_size=batch_size,
                                                       directory=validation_dir,
                                                       target_size=(IMG_SHAPE,IMG_SHAPE), #(150,150)
                                                       class_mode='categorical')

In [ ]:
test_data_gen = test_image_generator.flow_from_directory(batch_size=batch_size,
                                                         directory=test_dir,
                                                         target_size=(IMG_SHAPE,IMG_SHAPE), #(150,150)
                                                         class_mode='categorical')

### Visualizing Training images

We can visualize our training images by using following lines of code which will first extract a batch of images from training generator, which is 32 images in our case and then we will plot 5 of them using **matplotlib**


In [ ]:
sample_training_images, _ = next(train_data_gen)

**next** function returns a batch from the dataset. The return value of **next** function is in form of (x_train, y_train) where x_train is training features and y_train, its labels. We are discarding the labels in above situation because we only want to visualize our training images.

In [ ]:
# This function will plot images in the form of a grid with 1 row and 5 columns where images are placed in each column.
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip(images_arr, axes):
        ax.imshow(img)
    plt.tight_layout()
    plt.show()

In [ ]:
plotImages(sample_training_images[:5])

# Model Creation

The model consists of 3 convolution blocks with max pool layer in each of them. We have a fully connected layer with 512 units on top of it, which is activated by **relu** activation function. Model will output class probabilities based on categorical classification which is done by **softmax** activation function. 

In [ ]:
model = Sequential()
model.add(Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_SHAPE,IMG_SHAPE, 3,))) 
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, 3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, 3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(5, activation='softmax'))

### Compiling the model

We will use **ADAM** optimizer as our choice of optimizer for this task and **categorical cross entropy** function as a loss function. We would also like to look at training and validation accuracy on each epoch as we train our network, for that we are passing it in the metrics argument.

In [ ]:
# model.compile(optimizer='adam', 
#               loss='categorical_crossentropy', 
#               metrics=['accuracy'])

model.compile(optimizer=tf.train.AdamOptimizer(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

### Model Summary
Let's look at all the layers of our network using **summary** method.

In [ ]:
model.summary()

### Train the model

Its time we train our network. We will use **fit_generator** function to train our network instead of **fit** function, as we are using **ImageDataGenerator** class to generate batches of training and validation data for our network. 

In [ ]:
history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=int(np.ceil(num_train / float(batch_size))),
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=int(np.ceil(num_val / float(batch_size)))
)

### Visualizing results of the training

Let us now visualize the results we get after training our network.

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

As we can see from the plots, training accuracy and validation accuracy are off by large margin and our model has achieved only around **70%** accuracy on the validation set, let us analyse what went wrong there and try to increase overall performance of the model.

# Data Augmentation

Overfitting generally occurs when we have small number of training examples. One way to fix this problem is to augment our dataset so that it has sufficient number of training examples. Data augmentation takes the approach of generating more training data from existing training samples, by augmenting the samples via a number of random transformations that yield believable-looking images. The goal is that at training time, your model will never see the exact same picture twice. This helps expose the model to more aspects of the data and generalize better.

In **tf.keras** we can implement this using the same **ImageDataGenerator** class we used before. We can simply pass  different transformations we would want to our dataset as a form of arguments and it will take care of applying it to the dataset during our training process. 

### Applying Horizontal Flip

In [ ]:
image_gen = ImageDataGenerator(rescale=1./255, horizontal_flip=True)

In [ ]:
train_data_gen = image_gen.flow_from_directory(batch_size=batch_size,
                                               directory=train_dir,
                                               shuffle=True,
                                               target_size=(IMG_SHAPE,IMG_SHAPE))

In [ ]:
augmented_images = [train_data_gen[0][0][0] for i in range(5)]

In [ ]:
# Here, we are simply re-using the same custom plotting function 
# we defined and used above to visualize our training images
plotImages(augmented_images)

### Randomly rotating the image

Let's take a look at different augmentation called rotation and apply 45 degrees of rotation randomly to our training examples. 

In [ ]:
image_gen = ImageDataGenerator(rescale=1./255, rotation_range=45)

In [ ]:
train_data_gen = image_gen.flow_from_directory(batch_size=batch_size,
                                               directory=train_dir,
                                               shuffle=True, 
                                               target_size=(IMG_SHAPE, IMG_SHAPE))

augmented_images = [train_data_gen[0][0][0] for i in range(5)]

In [ ]:
plotImages(augmented_images)

### Applying Zoom

Let's apply Zoom augmentation to our dataset to zoom images up to 50% randomly.

In [ ]:
image_gen = ImageDataGenerator(rescale=1./255, zoom_range=0.5)

In [ ]:
train_data_gen = image_gen.flow_from_directory(batch_size=batch_size,
                                               directory=train_dir,
                                               shuffle=True,
                                               target_size=(IMG_SHAPE, IMG_SHAPE))

augmented_images = [train_data_gen[0][0][0] for i in range(5)]

In [ ]:
plotImages(augmented_images)

### Putting it all together

We can apply all the augmentations we saw above and even more with just one line of code. We can simply pass the augmentations as arguments with proper values and that would be all.

Here, we have applied rescale, rotation of 45 degrees, width shift, height shift, horizontal flip and zoom augmentation to our training images.

In [ ]:
image_gen_train = ImageDataGenerator(rescale=1./255,
                                     rotation_range=45,
                                     width_shift_range=.15,
                                     height_shift_range=.15,
                                     horizontal_flip=True,
                                     zoom_range=0.5)

In [ ]:
train_data_gen = image_gen_train.flow_from_directory(batch_size=batch_size,
                                                     directory=train_dir,
                                                     shuffle=True,
                                                     target_size=(IMG_SHAPE,IMG_SHAPE),
                                                     class_mode='categorical')

Let's visualize how a single image would look like 5 different times, when we pass these augmentations randomly to our dataset. 

In [ ]:
augmented_images = [train_data_gen[0][0][0] for i in range(5)]
plotImages(augmented_images)

## Evaluation for Test dataset

In [ ]:
history = model.evaluate_generator(test_data_gen)

In [ ]:
# loss
print("loss value: {:.3f}".format(history[0]))
# accuracy
print("accuracy value: {:.3f}".format(history[1]))